In [105]:
import requests
from bs4 import BeautifulSoup


In [189]:
URL = "https://www.mon-horoscope-du-jour.com/horoscopes/quotidien/poissons.htm"
page = requests.get(URL)

# print(page.text)
soup = BeautifulSoup(page.content, "html.parser")


#print(soup.prettify())
trucs = soup.find_all('span',class_="horo_content_stof")
titles = soup.find_all('h3')
paragraphes  = soup.find_all('div' , class_="horo_content_c")
#paragraphes = paragraphes.find_all('p',class_="horo_content_c")
#print(trucs [3].prettify())

In [191]:
for paragraphe in paragraphes:
    paragraphe = paragraphe.find('p')
    print('•' +  paragraphe.text)

i = 0
for truc in trucs:
    print(titles[i].text)
    i = i +1
    if truc.text == "★":
        print("1/5")
    elif truc.text == "★★":
        print("2/5")
    elif truc.text == "★★★":
        print("3/5")
    elif truc.text == "★★★★":
        print("4/5")
    else:
        print("5/5")
        

•Ce sont vos enfants, si vous en avez qui vous donneront entière satisfaction durant cette journée. Grâce à eux vous trouverez de la motivation. La journée devrait vous apporter son lot de bonnes surprises. Vos énergies sont à la hausse.
•Une fringale affective pourrait vous tenter au cours de cette journée. L'amour ou à défaut les possessions pourraient s'activer en votre esprit à l'affût des bonnes choses. « Faute de grive on mangera du merle » votre devise aujourd'hui !
•Vous prenez d'excellentes décisions en ce moment et vous enrichissez votre patrimoine avec beaucoup de sérieux. Votre entêtement passe pour de la maturité et pousse les investisseurs à vous suivre dans vos projets, c'est l'occasion de prouver votre efficacité.
•Cette journée est parfaite à l'éclosion de vos potentiels créatifs, à l'expression concrète de vos capacités, pour commencer un projet nouveau ou au contraire, en clôturer d'autres qui en ont besoin. Les astres sont avec vous profitez-en !
•Durant vos temps d

In [101]:
reqText1 = soup.find('p').text
print(reqText1)

La Lune en Taureau et Vénus en Scorpion vous font chercher la perle rare ! La réussite de votre vie affective compte. Écoutez votre intuition pour choisir.
